In [15]:
import numpy as np
from sklearn.datasets import *
from sklearn import neighbors
import matplotlib.pyplot as plt
import _csv
%matplotlib inline

In [18]:
with open("features.txt", 'r') as f:
    features = list(_csv.reader(f, delimiter=" "))
with open("labels.txt", 'r') as f:
    labels = list(_csv.reader(f, delimiter=" "))

#prune off excess data, clean up format of the data
del features[0]
del labels[0]
for i in range(len(features)):
    del features[i][0]
for i in range(len(features)):
    for j in range(len(features[i])):
        features[i][j] = float(features[i][j])
features = np.array(features).T.tolist()  # transpose the matrix to make it easier to read

temp_labels = labels
success = 0
for t in range(1, 10):
    temp_features=[]
    for i in range(len(features)):
        temp_row=[]
        for j in features[i]:
            temp_row.append(j)
        temp_features.append(temp_row)
    test_indices = []
    filename = "experiments/fold" + str(t) + "_test.txt"
    with open(filename, 'r') as f:
        test_indices = list(_csv.reader(f, delimiter=","))
    test_indices = [x[0] for x in test_indices]
    test_indices = [int(x) for x in test_indices]

    # divide the feature data into training and testing samples
    features_test = []

    for i in range(len(features)):
        test_row = []
        a = sum(features[i]) / len(features[i])  # average of features[i]
        for j in range(len(features[i])):
            features[i][j] /= a
            if j in test_indices:
                test_row.append(features[i][j])
                features[i][j] = -1
        features_test.append(test_row)
    for i in range(len(features)):
        features[i] = [x for x in features[i] if x != -1]
    feature_Arr_train = np.array(features).T
    feature_Arr_test = np.array(features_test).T

    # divide the label data into training and testing samples
    level_test = []
    levels = [labels[x][1] for x in range(len(labels))]
    for i in range(len(levels)):
        if i in test_indices:
            level_test.append(levels[i])
            levels[i] = -1
    levels = [x for x in levels if x != -1]
    level_Arr_train = np.array(levels)
    level_Arr_test = np.array(level_test)

    # fit the model to the two parameters

    model = neighbors.KNeighborsClassifier(n_neighbors=len(level_Arr_train), weights='distance')

    model.fit(feature_Arr_train, level_Arr_train)
    # check for accuracy
    for i in range(len(level_Arr_test)):
        # print feature_Arr_test[i]
        test_point = np.reshape(feature_Arr_test[i], (1, 8))
        # print test_point
        if model.predict(test_point)[0] == level_Arr_test[i]:
            success += 1
    features = temp_features
    labels = temp_labels
print success / float(36)

0.694444444444
